# Evolution of Cooperation

A reimplementation of [The Evolution of Trust](https://ncase.me/trust/) to familiarize myself with [Mesa](https://mesa.readthedocs.io/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from abc import ABC, abstractmethod
from enum import Enum
from operator import attrgetter
from itertools import combinations
from copy import copy
from collections import Counter, defaultdict, deque
import logging
import sys
import random

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.batchrunner import batch_run

logging.basicConfig(format='%(message)s',
                    stream=sys.stdout,
                    filemode='a',
                    level=logging.DEBUG,
                    datefmt='%Y-%m-%d %H:%M:%S%Z',
                    force=True)

## Actions and Consequences

In [2]:
class Action(Enum):
    COOPERATE = 1
    DEFECT = 2
    # Alises for occasional brevity
    C = 1
    D = 2

In [3]:
payoff_matrix = {
    (Action.COOPERATE, Action.COOPERATE):  (2, 2),
    (Action.COOPERATE, Action.DEFECT):    (-1, 3),
    (Action.DEFECT,    Action.COOPERATE):  (3, -1),
    (Action.DEFECT,    Action.DEFECT):     (0, 0),
}

## 'Player' Agent superclass

In [4]:
class Player(Agent, ABC):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 0
        self._init()
    
    def _init(self):
        """For any player-specific setup"""
        pass
        
    @abstractmethod
    def action(self, other) -> Action:
        """Return an action against another player"""
        pass
    
    def reaction(self, other, action: Action):
        """Process the other player's action"""
        pass
    
    def step(self):
        self._move()
        
    def _move(self):
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        choice = self.random.choice(neighborhood)
        self.model.grid.move_agent(self, choice)
    
    def __str__(self):
        return f'[{type(self).__name__}#{self.unique_id} wealth={self.wealth}]'

In [5]:
def play(left: Player, right: Player):
    """Play one match between two players"""
    left_a  = left.action(right)
    right_a = right.action(left)
    
    # inform the players of their opponents' actions
    left.reaction(right, right_a)
    right.reaction(left, left_a)
    
    (left_reward, right_reward) = payoff_matrix[(left_a, right_a)]
    left.wealth += left_reward
    right.wealth += right_reward
    
    logging.debug(f'[Match: [M: {left_a.name:>9} /{right_a.name:>10}]'
                 f' :: [R: {left_reward:>1}/{right_reward:>1}]'
                 f' :: [W: {left.wealth}/{right.wealth}]]')
    return (left_a, right_a)  # unused, except perhaps later for tests

In [6]:
def test_game(left_cls, right_cls):
    """Instantiate and play a test game against two player types"""
    mock_model = Model()
    this = left_cls(1, mock_model)
    other = right_cls(2, mock_model)

    print(f'Match between {this} and {other}')
    for _ in range(10):
        play(this, other)
    print(this)
    print(other)

## Players

In [7]:
class AlwaysCooperate(Player):
    def action(self, other):
        return Action.COOPERATE

In [8]:
class AlwaysDefect(Player):
    def action(self, other):
        return Action.DEFECT

In [9]:
class Copycat(Player):
    """First cooperates, then copies your last move"""
    
    def _init(self):
        self.prev_action = defaultdict(lambda: Action.COOPERATE)
    
    def action(self, other):
        return self.prev_action[other.unique_id]
    
    def reaction(self, other, opponent_action):
        self.prev_action[other.unique_id] = opponent_action

test_game(Copycat, AlwaysDefect)

Match between [Copycat#1 wealth=0] and [AlwaysDefect#2 wealth=0]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Copycat#1 wealth=-1]
[AlwaysDefect#2 wealth=3]


In [10]:
class Grudger(Player):
    """Cooperates until a defection, then defects forever"""
    
    def _init(self):
        self.grudge = defaultdict(lambda: Action.COOPERATE)
    
    def action(self, other):
        return self.grudge[other.unique_id]
    
    def reaction(self, other, opponent_action):
        if opponent_action is Action.DEFECT:
            self.grudge[other.unique_id] = Action.DEFECT

test_game(Grudger, AlwaysDefect)

Match between [Grudger#1 wealth=0] and [AlwaysDefect#2 wealth=0]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Grudger#1 wealth=-1]
[AlwaysDefect#2 wealth=3]


In [11]:
class Detective(Player):
    """Plays C D C C, then if you defected it becomes Copycat, otherwise AlwaysDefect"""
    def _init(self):
        self.next_actions = defaultdict(lambda: deque((Action.C, Action.D, Action.C, Action.C)))
        self.done_detecting = defaultdict(lambda: False)
        self.ever_defected = defaultdict(lambda: False)
    
    def action(self, other):
        do_this = self.next_actions[other].popleft()
        if len(self.next_actions[other]) == 0:
            self.done_detecting[other] = True
        return do_this
    
    def reaction(self, other, opponent_action):
        if opponent_action is Action.DEFECT:
                self.ever_defected[other] = True
        if self.done_detecting[other]:
            if self.ever_defected[other]:
                self.next_actions[other].append(opponent_action) # Copycat
            else:
                self.next_actions[other].append(Action.DEFECT) # AlwaysDefect
        
test_game(Detective, AlwaysDefect)

Match between [Detective#1 wealth=0] and [AlwaysDefect#2 wealth=0]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: -1/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -1/3]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: -2/6]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: -3/9]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -3/9]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -3/9]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -3/9]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -3/9]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -3/9]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: -3/9]]
[Detective#1 wealth=-3]
[AlwaysDefect#2 wealth=9]


In [12]:
class Copykitten(Player):
    """Like Copycat, but requires two defections before it defects"""
    
    def _init(self):
        self.prevs = defaultdict(lambda: deque((Action.COOPERATE, Action.COOPERATE), 2))
    
    def action(self, other):
        d = self.prevs[other.unique_id]
        if d[0] is d[1] is Action.DEFECT:
            return Action.DEFECT
        else:
            return Action.COOPERATE
    
    def reaction(self, other, opponent_action):
        self.prevs[other.unique_id].append(opponent_action)

test_game(Copykitten, Detective)

Match between [Copykitten#1 wealth=0] and [Detective#2 wealth=0]
[Match: [M: COOPERATE / COOPERATE] :: [R: 2/2] :: [W: 2/2]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 1/5]]
[Match: [M: COOPERATE / COOPERATE] :: [R: 2/2] :: [W: 3/7]]
[Match: [M: COOPERATE / COOPERATE] :: [R: 2/2] :: [W: 5/9]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 4/12]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 3/15]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 3/15]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 3/15]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 3/15]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 3/15]]
[Copykitten#1 wealth=3]
[Detective#2 wealth=15]


In [13]:
class Simpleton(Player):
    """Cooperates, then cooperation causes repeat, but Defect causes it to toggle"""
    
    def _init(self):
        self.do_this = defaultdict(lambda: Action.COOPERATE)
    
    def action(self, other):
        return self.do_this[other]
    
    def reaction(self, other, opponent_action):
        if opponent_action is Action.DEFECT:
            self.do_this[other] = Action.D if self.do_this[other] is Action.C else Action.C

test_game(Simpleton, Detective)

Match between [Simpleton#1 wealth=0] and [Detective#2 wealth=0]
[Match: [M: COOPERATE / COOPERATE] :: [R: 2/2] :: [W: 2/2]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 1/5]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 4/4]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 7/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 7/3]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 6/6]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 9/5]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 9/5]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 8/8]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 11/7]]
[Simpleton#1 wealth=11]
[Detective#2 wealth=7]


In [14]:
class Random(Player):
    """Plays a random move (50/50) each time"""
    
    def _init(self):
        self.actions = list(Action)
    
    def action(self, other):
        return self.random.choice(self.actions)

test_game(Random, Detective)

Match between [Random#1 wealth=0] and [Detective#2 wealth=0]
[Match: [M: COOPERATE / COOPERATE] :: [R: 2/2] :: [W: 2/2]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 1/5]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 4/4]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 7/3]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 7/3]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 6/6]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 9/5]]
[Match: [M: COOPERATE /    DEFECT] :: [R: -1/3] :: [W: 8/8]]
[Match: [M:    DEFECT / COOPERATE] :: [R: 3/-1] :: [W: 11/7]]
[Match: [M:    DEFECT /    DEFECT] :: [R: 0/0] :: [W: 11/7]]
[Random#1 wealth=11]
[Detective#2 wealth=7]


In [15]:
player_types = [AlwaysCooperate, AlwaysDefect, Copycat, Grudger, Detective, Copykitten, Simpleton, Random]

## Model

In [16]:
def agent_types_breakdown(model: Model):
    class_names = [type(a).__name__ for a in model.schedule.agents]
    return Counter(class_names)

In [17]:
class TrustModel(Model):
    def __init__(self, N, width, height):
        self.N = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True
        self.datacollector = DataCollector(
            model_reporters={"Breakdown": agent_types_breakdown}
        )
        self.current_id = 0  # apparently necessary to make next_id() work
        
        for i in range(self.N):
            a = player_types[i%len(player_types)](self.next_id(), self)
            self.schedule.add(a)
            self.grid.place_agent(a, self.randpos())
            
    def randpos(self):
        x = self.random.randrange(self.grid.width)
        y = self.random.randrange(self.grid.height)
        return (x,y)
            
    def step(self):
        self.datacollector.collect(self)
        self.reset_wealth()
        self.schedule.step()
        self.run_tournaments()
        self.kill_underperformers()
        self.clone_outperformers()
        
    def run_tournaments(self):
        for (contents, x, y) in model.grid.coord_iter():
            pairs = combinations(contents, r=2)
            for pair in pairs: # for every possible pair in the cell
                for i in range(10): # play ten games
                    play(*pair)
    
    def kill_underperformers(self):
        for a in sorted(self.schedule.agents, key=attrgetter('wealth'))[:5]:
            self.grid.remove_agent(a)
            self.schedule.remove(a)
    
    def clone_outperformers(self):
        for a in sorted(self.schedule.agents, key=attrgetter('wealth'), reverse=True)[:5]:
#             a_copy = copy(a)
            a_copy = type(a)(self.next_id(), self)
#             a_copy.unique_id = self.next_id()
            self.schedule.add(a_copy)
            self.grid.place_agent(a_copy, self.randpos())
    
    def reset_wealth(self):
        for a in self.schedule.agents:
            a.wealth = 0

## Run it

In [18]:
logging.basicConfig(format='%(message)s',
                    stream=sys.stdout,
                    filemode='a',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S%Z',
                    force=True)

In [19]:
model = TrustModel(50, 1, 1) # for now, it all plays out on one grid cell only

for i in range(50):
    model.step()

In [20]:
breakdown = model.datacollector.get_model_vars_dataframe()
breakdown.style

,Breakdown
0,"Counter({'AlwaysCooperate': 7, 'AlwaysDefect': 7, 'Copycat': 6, 'Grudger': 6, 'Detective': 6, 'Copykitten': 6, 'Simpleton': 6, 'Random': 6})"
1,"Counter({'Copycat': 11, 'AlwaysCooperate': 7, 'Grudger': 6, 'Detective': 6, 'Copykitten': 6, 'Simpleton': 6, 'Random': 5, 'AlwaysDefect': 3})"
2,"Counter({'Copycat': 16, 'AlwaysCooperate': 7, 'Grudger': 6, 'Detective': 6, 'Copykitten': 6, 'Simpleton': 6, 'Random': 3})"
3,"Counter({'Copycat': 21, 'AlwaysCooperate': 7, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'Detective': 4})"
4,"Counter({'Copycat': 26, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'AlwaysCooperate': 6})"
5,"Counter({'Copycat': 26, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'AlwaysCooperate': 6})"
6,"Counter({'Copycat': 26, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'AlwaysCooperate': 6})"
7,"Counter({'Copycat': 26, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'AlwaysCooperate': 6})"
8,"Counter({'Copycat': 26, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'AlwaysCooperate': 6})"
9,"Counter({'Copycat': 26, 'Grudger': 6, 'Copykitten': 6, 'Simpleton': 6, 'AlwaysCooperate': 6})"
